In [18]:
# uncomment below to pip install the dependencies
# !python -m pip install pandas
# !python -m pip install scikit-learn


# optional dependency which is required for reading parquet files in pandas 
# uncomment to install...
#!python -m pip install pyarrow


# imports...
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score

In [8]:
# load the data into a pandas dataframe, drop the last column
data = pd.read_parquet('./data/flight_data.parquet', engine='pyarrow').iloc[:,:-1]

# drop ArrDelay (1-1 relation with target variable) and 'Carrier' - optionally we could instead encode 'Carrier' with ".astype('category').cat.codes"
data.drop('Carrier', axis=1, inplace=True)
data.drop('ArrDelay', axis=1, inplace=True)

# drop rows containing Na, print first 5 rows
data = data.dropna()
data.head()

,Year,Month,DayofMonth,DayOfWeek,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDel15
0,2013,4,19,5,11433,13303,837,-3.0,0.0,1138,0.0
1,2013,4,19,5,14869,12478,1705,0.0,0.0,2336,0.0
2,2013,4,19,5,14057,14869,600,-4.0,0.0,851,0.0
3,2013,4,19,5,15016,11433,1630,28.0,1.0,1903,1.0
4,2013,4,19,5,11193,12892,1615,-6.0,0.0,1805,0.0


In [9]:
# We only use 10k rows for training as it is enough for the example. 
# You don't want your output model to be too large, and in our case, reducing the training data also decreases the model size..
print(r"WARNING: Dropping most of the data of data")
data = data[:10000]
print(data.shape)

(10000, 11)


In [13]:
# split the data into predictors and target variable, and split the data. 
x, y = data.iloc[:,:-1], data.iloc[:,-1]
xtrain, xtest, ytrain, ytest = train_test_split(x, y)

In [16]:
# create the classifier object, fit the classifier with the training data
clf = RF()
clf.fit(xtrain, ytrain)

RandomForestClassifier()

In [21]:
# Sanity check: check the model accuracy on the test subset
y_pred_test = clf.predict(xtest)
print(accuracy_score(ytest, y_pred_test))

0.9204


In [22]:
# save the model
joblib.dump(clf, "./model/model.joblib")

['./model/model.joblib']